In [1]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

### MNIST con K-vecinos más vercanos
En esta segunda sesión práctica vamos a entrenar clasificadores basados en distancias.


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)



/home/joan/anaconda3/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [3]:
# Partición de los datos

x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size = 0.2, random_state= 23)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state=23)


### **Ejercicio 2**   

Prueba un clasificador KNeighborsClassifier

In [ ]:
kne = KNeighborsClassifier()

kne.fit(x_train, y_train)
y_pred = kne.predict(x_val)
acc = accuracy_score(y_pred, y_val)
print("la precision es :", acc)

In [4]:
## Prueba el clasificador KNeighborsClassifier

kne = KNeighborsClassifier()

kne.fit(x_train, y_train)
y_pred = kne.predict(x_test)
acc = accuracy_score(y_pred, y_test)
print("la precision es :", acc)

la precision es : 0.9734285714285714


### **Ejercicio 3**   

Encuentra el mejor valor de "K" con el conjunto de validación y obtén el resultados para test.

In [6]:
## Estimar el mejor parámetro K con el conjunto de validación

for i in [1,2,3,4,5,6,7,8,9,10,20]:    
    kne = KNeighborsClassifier(n_neighbors=i)

    kne.fit(x_train, y_train)
    y_pred = kne.predict(x_val)
    acc = accuracy_score(y_pred, y_val)
    print("la precision para", i,"vecinos es :", acc)

la precision para 1 vecinos es : 0.9721428571428572
la precision para 2 vecinos es : 0.9672857142857143
la precision para 3 vecinos es : 0.9724285714285714
la precision para 4 vecinos es : 0.9711428571428572
la precision para 5 vecinos es : 0.9714285714285714
la precision para 6 vecinos es : 0.9724285714285714
la precision para 7 vecinos es : 0.9697142857142858
la precision para 8 vecinos es : 0.9681428571428572
la precision para 9 vecinos es : 0.9675714285714285
la precision para 10 vecinos es : 0.968
la precision para 20 vecinos es : 0.9605714285714285


In [4]:
## Obtener tasa de acierto con el conjunto de test para el mejor K
x_train_comp =np.concatenate((x_train, x_val), axis =0)

y_train_comp = np.concatenate((y_train, y_val))

kne = KNeighborsClassifier(n_neighbors=6 )

kne.fit(x_train, y_train)
y_pred = kne.predict(x_test)
acc = accuracy_score(y_pred, y_test)
print("la precision para 6 vecinos es :", acc)



la precision para 6 vecinos es : 0.9731428571428572


In [ ]:
## Obtener tasa de acierto con el conjunto de test para el mejor K

## **Ejercicio 4**

Cambia la distancia que se emplea (por defecto la L2) por la L1. Ver la documentación para ver cómo poder definir funciones de distancia y pasarlas como parámetro. Esto ralentiza bastante la clasificación pero se puede emplear el parámetro "n_jobs" para paralelizar el proceso. Aún así el resultado es muy lento. Podríamos concluir que sk-learn no es una librería adecuada para clasificar con KNN con métricas definidas por el usuario. Por ejemplo no permite definir un algoritmo rápido de tipo kd_tree.

In [5]:
## Modifica la función de distancia 
## x_test[0:100,:]

def distancia(x,y):
    return np.sum(np.abs(x-y))

kne = KNeighborsClassifier(n_neighbors=6, metric=distancia, n_jobs=4)

kne.fit(x_train_comp, y_train_comp)
y_pred = kne.predict(x_test)
acc = accuracy_score(y_pred, y_test)
print("la precision para 6 vecinos es :", acc)

In [ ]:
def distancia(x,y):
    return np.sum(np.abs(x-y))

kv = KNeighborsClassifier(n_neighbors=3,metric=distancia,n_jobs=4)
kv.fit(x_train_comp,y_train_comp)

print("predict")
ypred = kv.predict(x_test[0:100,:])
acc = accuracy_score(y_test[0:100], ypred[0:100])
print(acc)

In [ ]:
for n in [2,4,8,16,32,64]:
    for md in [3, 4, 6, 8, 10, 12]:
        
        clf = make_pipeline(PCA(n_components=n), KNeighborsClassifier(n_neighbors=md))
        
        clf.fit(x_train,y_train)
        ypred = clf.predict(x_val)
        acc = accuracy_score(y_val, ypred)
        print(acc)
        

In [ ]:
data_tr = np.concatenate((x_train, x_val))
targets_tr = ((y_train, y_val))
clf = make_pipeline(PCA(n_components=n), KNeighborsClassifier(n_neighbors=md))

clf.fit(data_tr,targets_tr)
ypred = clf.predict(x_test)
acc = accuracy_score(y_test, ypred)
print(acc)